In [33]:
import re

# --- STEP 1: A FULLY STOPWORD SET ---

NEPALI_STOPWORDS = {
    # Pronouns & Determiners
    'म', 'हामि', 'तँ', 'तिमि', 'तपाई', 'उ', 'उनि', 'उहाँ', 'यो', 'त्यो', 'यि', 'ति',
    'मेरो', 'तिम्रो', 'तपाईको', 'हाम्रो', 'उसको', 'उनको', 'उहांको', 'यसको', 'त्यसको',
    'मलाई', 'हामिलाई', 'तिमीलाई', 'तपाईलाई', 'उसलाई', 'उनलाई', 'उहाँलाई',
    'आफु', 'आफ्नो', 'आफै', 'जो', 'जसले', 'जुन', 'कुनै', 'केहि', 'सबै', 'हरेक',
    'कोहि', 'कसैले', 'कसैलाई', 'आफुलाई', 'आफ्ना', 'उक्त', 'सोहि', 'त्यहि', 'यहि',

    # Conjunctions & Particles
    'र', 'अनि', 'पनि', 'वा', 'तर', 'तथा', 'कि', 'भने', 'जब', 'तब', 'यदि', 'भन्दै', 'भन्ने',
    'नै', 'त', 'नि', 'रे', 'खै', 'पो', 'है', 'एवं', 'समेत', 'साथै', 'वा', 'अर्थात',

    # Postpositions (Case Markers)
    'को', 'का', 'कि', 'रो', 'रा', 'रि', 'नो', 'ना', 'नि',
    'मा', 'बाट', 'देखि', 'सम्म', 'तिर', 'तर्फ', 'संग', 'बिना', 'भित्र', 'बाहिर',
    'माथि', 'तल', 'बीच', 'निम्ति', 'लागि', 'ले', 'लाई', 'द्वारा',

    # Common Verbs (Auxiliary and simple forms)
    'छ', 'छन', 'छौ', 'छु', 'छैन', 'छैनन', 'छैन्',
    'हो', 'होइन', 'होला', 'हुन', 'हुने', 'हुन्छ', 'हुदैन',
    'भयो', 'भएको', 'भए', 'भएका', 'भएर', 'भइ', 'भइसकेको', 'भएपछि', 'भएकाले', 'भइरहेको',
    'थियो', 'थिए', 'थिइन', 'थिएन',
    'गर्नु', 'गरेको', 'गर्यो', 'गरि', 'गर्दै', 'गर्ने', 'गरे', 'गरेका', 'गरिएको', 'गरेर', 'गर्दा', 'गर्न',
    'दिनु', 'दिने', 'दियो', 'दिएको', 'दिए', 'दिदै',
    'लिनु', 'लिने', 'लियो', 'लिएको', 'लिएर',
    'हुनु', 'हुंदा', 'हुनाले',
    'रहेको', 'रहेका', 'रहेछ',
    'जानु', 'जाने', 'गएको', 'गए', 'गइ',
    'आउनु', 'आउने', 'आएको', 'आए', 'आएर',
    'पर्नु', 'पर्ने', 'परेको', 'परे',
    'खानु', 'खाने',
    'लाग्नु', 'लाग्ने', 'लाग्यो', 'लागेको',
    'सक्नु', 'सक्ने', 'सकिने', 'सक्छ',
    'बताए', 'बताउनु', 'भन्नु', 'भएछ',

    # Common Adverbs, Adjectives & Quantifiers
    'आज', 'भोलि', 'हिजो', 'अहिले', 'अब', 'फेरि', 'सधै', 'यहाँ', 'त्यहाँ', 'कहाँ',
    'यसरि', 'त्यसरि', 'कसरि', 'धेरै', 'थोरै', 'अलि', 'निकै', 'एकदम', 'मात्र', 'मात्रै',
    'पटक', 'दिन', 'रात', 'बर्ष', 'महिना', 'हप्ता', 'गते', 'पहिले', 'पछि', 'अघि', 'वरिपरि',
    'अनुसार', 'बमोजिम', 'उक्त', 'सो', 'एक', 'दुई', 'तिन', 'चार', 'पाँच',
    'राम्रो', 'नराम्रो', 'ठुलो', 'सानो', 'नयाँ', 'पुरानो', 'हरेक', 'प्रत्येक',
    'करिब', 'झन्डै', 'लगभग', 'आजको', 'भोलिको', 'यस', 'त्यस', 'संगै',

    # Question Words
    'के', 'किन', 'कसले', 'कसलाई', 'कहिले', 'कसरि', 'कति', 'कुन',
}



# --- STEP 2: PRE-PROCESSING FUNCTION (Helper, Unchanged) ---
def normalize_word_for_check(word):
    """
    Normalizes a single word for checking against the stopword set.
    """
    word = word.lower()
    word = word.replace('ँ', 'ं')
    word = word.replace('ी', 'ि')
    word = word.replace('ू', 'ु')

    # Remove any stray punctuation that might be attached to the word
    punctuation_pattern = r'[।(),?!:"\'\/-]'
    word = re.sub(punctuation_pattern, '', word)
    return word

normalized_stopwords = [normalize_word_for_check(word) for word in NEPALI_STOPWORDS]

# --- STEP 3: REWRITTEN MAIN FUNCTION ---
def remove_nepali_stopwords(text, stopwords_set=normalized_stopwords):
    """
    Takes a raw Nepali text, removes stopwords while preserving the
    original spelling and punctuation of the remaining words.
    
    Args:
        text (str): The raw input Nepali text.
        stopwords_set (set): A set of normalized stopwords to remove.
        
    Returns:
        str: The text with stopwords removed, in its original form.
    """
    # Pattern to find punctuation and split text around it, keeping the punctuation
    # This ensures punctuation marks are treated as separate tokens.
    tokenization_pattern = r'([।(),?!:"\'\/-]|\s+)'
    
    # Split the text by the pattern, which separates words and punctuation
    original_tokens = re.split(tokenization_pattern, text)
    
    # List to hold the words/punctuation we want to keep
    final_tokens = []
    
    for token in original_tokens:
        # Ignore empty strings that can result from splitting
        if not token.strip():
            continue
            
        # Normalize the token just for the purpose of checking
        normalized_token = normalize_word_for_check(token)
        
        # If the normalized token is not in the stopword list, keep the ORIGINAL token
        if normalized_token not in stopwords_set:
            final_tokens.append(token+" ")
            
    # Join the final list of tokens back into a coherent string
    return "".join(final_tokens).strip()

# --- DEMONSTRATION OF THE CORRECTED LOGIC ---
if __name__ == "__main__":
    sample_text = "उहांको लागि चिया पनि 1,00 ल्याउनुहोस्, तरकारी मीठो छैन्। हामी सबै सँगै बस्छौं।"

    print(f"Original Text:\n'{sample_text}'")
    
    cleaned_text = remove_nepali_stopwords(sample_text)
    
    print("\n" + "="*50 + "\n")
    print(f"Final Text (Original Spelling Preserved):\n'{cleaned_text}'")

Original Text:
'उहांको लागि चिया पनि 1,00 ल्याउनुहोस्, तरकारी मीठो छैन्। हामी सबै सँगै बस्छौं।'


Final Text (Original Spelling Preserved):
'चिया 1 , 00 ल्याउनुहोस् , तरकारी मीठो । बस्छौं ।'
